In [1]:
from sympy import *
from sympy.tensor import *


from variables import *
from structure import *
from functions import *
from latex     import *

In [2]:
def subs_delta(eq):
    return lambda i: eq(i).subs(
        Derivative(alpha,t), delta['alpha']).subs(
        Derivative(psi[i],t), delta['psi'][i]).subs(
        Derivative(theta[i],t), delta['theta'][i])

In [3]:
# угловые скорости (платформы, вилки, колеса) относительно пола
omega['platform']= lambda i: Derivative(alpha,t)*e['z']
omega['fork']    = lambda i: omega['platform'](i) + Derivative(theta[i],t)*e['z']
omega['wheel']   = lambda i: omega['fork'](i) + Derivative(psi[i],t)*n_wheel(i)

In [4]:
# УРАНЕНИЯ ЭЁЛЕРА И ОТСУТСТВИЕ ПРОСКАЛЬЗЫВАНИЯ 
v[fkey(S)] = euler(S, P)
v[fkey(P)] = euler(P, C)
v[fkey(C)] = euler(C, D)
v[fkey(D)] = lambda i: Matrix([0,0,0]) # проскальзывания нет

In [5]:
#Полученные выражения из связей для nu1 nu2 delta_x delta_y
eq[fkey(delta['x'])] = lambda i: scalar(v[fkey(S)](i), e['x'])
eq[fkey(delta['y'])] = lambda i: scalar(v[fkey(S)](i), e['y'])
eq[fkey(nu[1])]      = lambda i: scalar(v[fkey(S)](i), e['xi'])
eq[fkey(nu[2])]      = lambda i: scalar(v[fkey(S)](i), e['eta'])

In [6]:
eq['f(delta_x,delta_y)'] = lambda i: solve(
                      [Eq(eq[fkey(delta['x'])](i), delta['x']), Eq(eq[fkey(delta['y'])](i), delta['y'])],
                      [Derivative(psi[i],t), Derivative(theta[i],t)],
                      dict=True)[0]; # возвращает словарь с выражениями для diff(psi) и diff(theta)

eq['diff(psi)']   = lambda i: eq['f(delta_x,delta_y)'](i)[Derivative(psi[i],t)].subs(Derivative(alpha,t), delta['alpha'])
eq['diff(theta)'] = lambda i: eq['f(delta_x,delta_y)'](i)[Derivative(theta[i],t)].subs(Derivative(alpha,t), delta['alpha'])

In [32]:
eq['diff(theta)'](0)

(-d*delta_alpha - delta_alpha*cos(beta[0] - theta[0]) - delta_x*sin(alpha(t) + theta[0]) + delta_y*cos(alpha(t) + theta[0]))/d

In [7]:
# eq['f(nu1,nu2)'] = lambda i: solve(
#                       [Eq(eq[fkey(nu[1])](i), nu[1]), Eq(eq[fkey(nu[2])](i), nu[2])],
#                       [Derivative(psi[i],t), Derivative(theta[i],t)],
#                       dict=True)[0]; # возвращает словарь с выражениями для diff(psi) и diff(theta)
# eq['diff(psi)']   = lambda i: eq['f(delta_x,delta_y)'](i)[Derivative(psi[i],t)].subs(Derivative(alpha,t), delta['alpha'])
# eq['diff(theta)'] = lambda i: eq['f(delta_x,delta_y)'](i)[Derivative(theta[i],t)].subs(Derivative(alpha,t), delta['alpha'])

In [8]:
#ТЕЛО 1 (платформа)
velocity[fkey(S)]        = lambda i: nu[1]*e['xi'] + nu[2]*e['eta']  # !!!! (надо бы подставить, скорость которую уже считали выше) 
F['platform']            = lambda i: zeros(3,1) # сил не действует
delta_r[fkey(S)]         = subs_delta(lambda i: delta['x']*e['x'] + delta['y']*e['y'])
J['platform']            = eye(3,3)*a  # !!!!! (симметричная) (временные коэффиценты)
K['platform']            = lambda i: J['platform']*omega['platform'](i)
omega_delta['platform']  = subs_delta(lambda i: omega['platform'](i).subs(Derivative(alpha,t),delta['alpha']))
M['platform']            = lambda i: -W[0]*e['z']

A['platform'] = dalamber(mass = m['platform'],
                         velocity = velocity[fkey(S)],
                         F = F['platform'],
                         delta_r = delta_r[fkey(S)],
                         K = K['platform'],
                         M = M['platform'],
                         omega_delta = omega_delta['platform'])

In [9]:
#ТЕЛО 2 (вилки)
velocity[fkey(P)]    = lambda i: velocity[fkey(S)](i) + cross(omega['platform'](i), vec_by_2dots(S,P)(i))   # !!!!
# velocity['fork']   = v[fkey(P)]
F['fork']            = lambda i: zeros(3,1) # сил не действует
delta_r[fkey(P)]     = subs_delta(lambda i: delta_r[fkey(S)](i) + cross(omega['platform'](i), vec_by_2dots(S,P)(i)))
J['fork']            = zeros(3,3) # невесома
K['fork']            = lambda i: J['fork']*omega['fork'](i)
omega_delta['fork']  = subs_delta(lambda i: omega['fork'](i))
M['fork']            = lambda i: -(-W[i]*e['z']) -(-T[i]*n_wheel(i)) # magic

A['fork'] = dalamber(mass = 0,
                     velocity = velocity[fkey(P)],
                     F = F['fork'],
                     delta_r = delta_r[fkey(P)],
                     K = K['fork'],
                     M = M['fork'],
                     omega_delta = omega_delta['fork'])

In [10]:
#ТЕЛО 3 (колёса)
velocity[fkey(C)]     = lambda i: velocity[fkey(P)](i) + cross(omega['fork'](i), vec_by_2dots(P,C)(i))   # !!!!
F['wheel']            = lambda i: zeros(3,1) # сил не действует
delta_r[fkey(C)]      = subs_delta(lambda i: delta_r[fkey(P)](i) + cross(omega['fork'](i), vec_by_2dots(S,P)(i)))
J['wheel']            = Matrix([[a,0,0],[0,b,0],[0,0,c]])
K['wheel']            = lambda i: J['wheel']*omega['wheel'](i)
omega_delta['wheel']  = subs_delta(lambda i: omega['wheel'](i))
M['wheel']            = lambda i: -(-W[i]*e['z']) -(-T[i]*n_wheel(i)) # magic

A['wheel'] = dalamber(mass= m['wheel'],
                     velocity = velocity[fkey(C)],
                     F = F['wheel'],
                     delta_r = delta_r[fkey(C)],
                     K = K['wheel'],
                     M = M['wheel'],
                     omega_delta = omega_delta['wheel'])


In [11]:
# Полный Д'Аламбер Лагранж
A_full  = lambda i: A['platform'](i) + A['fork'](i) + A['wheel'](i)
A_full_ = lambda i: Poly(A_full(i).subs(delta['psi'][i], eq['diff(psi)'](i)).subs(delta['theta'][i], eq['diff(theta)'](i)), 
    [delta['x'], delta['y'], delta['alpha']])

In [35]:
A_full_(0)

Poly((-a*sin(alpha(t))**2*sin(theta[0])*cos(alpha(t) + theta[0])*cos(theta[0])*Derivative(alpha(t), t)*Derivative(psi[0], t)/r - a*sin(alpha(t))*sin(theta[0])**2*cos(alpha(t) + theta[0])*cos(alpha(t))*Derivative(alpha(t), t)*Derivative(psi[0], t)/r + a*sin(alpha(t))*cos(alpha(t) + theta[0])*cos(alpha(t))*cos(theta[0])**2*Derivative(alpha(t), t)*Derivative(psi[0], t)/r + a*sin(theta[0])*cos(alpha(t) + theta[0])*cos(alpha(t))**2*cos(theta[0])*Derivative(alpha(t), t)*Derivative(psi[0], t)/r + b*sin(alpha(t))**2*sin(theta[0])*cos(alpha(t) + theta[0])*cos(theta[0])*Derivative(alpha(t), t)*Derivative(psi[0], t)/r + b*sin(alpha(t))*sin(theta[0])**2*cos(alpha(t) + theta[0])*cos(alpha(t))*Derivative(alpha(t), t)*Derivative(psi[0], t)/r - b*sin(alpha(t))*cos(alpha(t) + theta[0])*cos(alpha(t))*cos(theta[0])**2*Derivative(alpha(t), t)*Derivative(psi[0], t)/r - b*sin(theta[0])*cos(alpha(t) + theta[0])*cos(alpha(t))**2*cos(theta[0])*Derivative(alpha(t), t)*Derivative(psi[0], t)/r - c*sin(alpha(t) + 

In [12]:
# Коэффиценты
A_coeffs = lambda i: A_full_(i).coeffs()

In [45]:
# Здесь будут лежать коэффиценты при dx dy dalpha  для всех 3 колёс
var = A_coeffs(0)

# coeff_wheel.append(A_coeffs(1))
# coeff_wheel.append(A_coeffs(2))

In [41]:
l = 0
#var[0] = var[0].subs({alpha: 0, beta[l]: 0, theta[l]: 0, nu[1]: 0, nu[2]: 0})
#var[1] = var[1].subs({alpha: 0, beta[l]: 0, theta[l]: 0, nu[1]: 0, nu[2]: 0})
#var[2] = var[2].subs({alpha: 0, beta[l]: 0, theta[l]: 0, nu[1]: 0, nu[2]: 0}) 

In [44]:
pprint(var)

⎡                                                                  2          
⎢                                                                 d           
⎢                                                              c⋅───(0)       
⎢                2                        2                        2          
⎢         ⎛d    ⎞       d          ⎛d    ⎞       d       T[0]    dt           
⎢- 2⋅d⋅m₂⋅⎜──(0)⎟  + m₁⋅──(0) - m₂⋅⎜──(0)⎟  + m₂⋅──(0) + ────, ──────── + d⋅m₂
⎢         ⎝dt   ⎠       dt         ⎝dt   ⎠       dt       r       d           
⎣                                                                             

                                                              2               
                                                             d                
                                                 d       m₂⋅───(0)            
   2                                   2      m₂⋅──(0)        2               
  d          d          d             d          dt

In [49]:
coeff[delta['x']]     = simplify(var[0])#.subs({alpha: 0, beta[i]: 0, theta[i]: 0, nu[1]: 0, nu[2]: 0})
coeff[delta['y']]     = simplify(var[1])#.subs({alpha: 0, beta[i]: 0, theta[i]: 0, nu[1]: 0, nu[2]: 0})
coeff[delta['alpha']] = simplify(var[2])#.subs({alpha: 0, beta[i]: 0, theta[i]: 0, nu[1]: 0, nu[2]: 0})

KeyboardInterrupt: 

In [ ]:
coeff[delta['x']]

In [38]:
#коэффиценты для колеса при dx dy dalpha
k = 0 # номер колеса
coeff_dx = coeff[delta['x']](k)
coeff_dy = coeff[delta['y']](k)
coeff_dalpha = coeff[delta['alpha']](k)

KeyboardInterrupt: 

In [41]:
# посмотреть что получилось
print(coeff_dx)

(d*r*(-2*d*m2*Derivative(0, t)**2 + m1*Derivative(0, t) - m2*Derivative(0, t)**2 + m2*Derivative(0, t)) + d*T[0])/(d*r)


In [29]:
# говорим что получившиеся выражения это полином, зависящий от nu1 nu2 (alpha)'' 
eq_by_dx     = Poly(coeff_dx,     [Derivative(nu[1],t), Derivative(nu[2],t), Derivative(alpha,t,2)])
eq_by_dy     = Poly(coeff_dy,     [Derivative(nu[1],t), Derivative(nu[2],t), Derivative(alpha,t,2)])
eq_by_dalpha = Poly(coeff_dalpha, [Derivative(nu[1],t), Derivative(nu[2],t), Derivative(alpha,t,2)])

In [33]:
# посмотреть что получилось
eq_by_dalpha.coeffs()

[-m2*sin(beta[0]) + m2*sin(beta[0])*cos(beta[0] - theta[0])/d,
 m2*cos(beta[0]) - m2*cos(beta[0] - theta[0])*cos(beta[0])/d,
 a - c*cos(beta[0] - theta[0])/d + d*m2*cos(beta[0] - theta[0]) - m2*cos(2*beta[0] - 2*theta[0])/2 + m2/2 - m2*cos(beta[0] - theta[0])/d,
 a*sin(2*alpha(t) + 2*theta[0])*sin(beta[0] - theta[0])*Derivative(alpha(t), t)*Derivative(psi[0], t)/(2*r) - b*sin(2*alpha(t) + 2*theta[0])*sin(beta[0] - theta[0])*Derivative(alpha(t), t)*Derivative(psi[0], t)/(2*r) + d*m2*sin(beta[0] - theta[0])*Derivative(alpha(t), t)**2 + d*m2*sin(beta[0] - theta[0])*Derivative(alpha(t), t)*Derivative(theta[0], t) + m2*nu1(t)*cos(beta[0])*Derivative(alpha(t), t) + m2*nu2(t)*sin(beta[0])*Derivative(alpha(t), t) - m2*sin(2*beta[0] - 2*theta[0])*Derivative(alpha(t), t)**2/2 - m2*sin(2*beta[0] - 2*theta[0])*Derivative(alpha(t), t)*Derivative(theta[0], t)/2 - W[0] + sin(beta[0] - theta[0])*T[0]/r - m2*nu1(t)*cos(beta[0] - theta[0])*cos(beta[0])*Derivative(alpha(t), t)/d - m2*nu2(t)*sin(beta[0])*

In [50]:
# система уравнений
pprint(Matrix([eq_by_dx.coeffs()[:3],
               eq_by_dy.coeffs()[:3],
               eq_by_dalpha.coeffs()[:3]]))

NameError: name 'eq_by_dx' is not defined